<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [76]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from keras.layers import Dense, Dropout, LSTM, BatchNormalization
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import sklearn.preprocessing as p

In [104]:
data = pd.read_csv("export_dataframe.csv") 
#need of an array with the id of the papers
data = data.sort_index(axis=1, level=None, ascending=True, inplace=False, kind='quicksort', na_position='last', sort_remaining=True, by=None)


In [3]:
headers = []

headers = data.columns
for i in range(0, 45):
    data = data.drop([headers[i]], axis = 1)

In [4]:
years = data
years = years.drop('ids', axis = 1)
years = years.drop('2018', axis = 1)
all_years = np.asarray(years)

In [5]:
l = len(all_years)
deleted_years = []
for i in range(l):
    if any(j != 0 for j in all_years[i]):
        deleted_years.append(all_years[i])

In [6]:
l = len(deleted_years)
smaller_array = []
for i in range(l):
    temp_count = 0
    for j in deleted_years[i]:
        temp_count = temp_count + j
        if temp_count > 90:
            smaller_array.append(deleted_years[i])


LSTMs are sensitive to the scale of the input data, specifically when the sigmoid or tanh activation functions are used. It’s generally a good practice to rescale the data to the range of [0, 1] or [-1, 1], also called normalizing.


In [8]:

from sklearn.preprocessing import MinMaxScaler
from pandas import Series

In [9]:
def window_transform_series(series, window_size):
    # containers for input/output pairs
    X,y = [],[]
    l = len(series)
    for idx in range(l - window_size):
        X.append(series[idx:idx+window_size])
    
    y = series[window_size:]
    
    # reshape each 
    X = np.asarray(X)
    X.shape = (np.shape(X)[0:2])
    y = np.asarray(y)
    y.shape = (len(y),1)
    
    
    return X,y

In [74]:
def model1():
    
    window_size = 5

    lstm_model.add(LSTM( 5, activation = 'relu', input_shape=(5, 1), return_sequences = True))
    lstm_model.add(LSTM( 5, activation = 'relu', return_sequences = False))
    lstm_model.add(Dropout(0.3))
    lstm_model.add(Dense(1))
    lstm_model.compile(optimizer='Adam',  metrics = ['accuracy'],loss='mse')

In [77]:
lstm_model = Sequential()
model1()

In [99]:
def prediction(X, y):
    #X = np.asarray(np.reshape(X, (X.shape[0], window_size, 1)))
    predict = lstm_model.predict(X)
    error = lstm_model.evaluate(X, y, verbose=0)
    acc = mean_squared_error(predict, y)
    return predict


In [91]:
for i in smaller_array:
    
    window_size = 5
    X, y = window_transform_series(i, window_size = window_size)

    train_test_split = int(np.ceil(2*len(y)/float(3)))   # set the split point
    


          # partition the training set
    X_train = X[:train_test_split,:]
    y_train = y[:train_test_split]

            # keep the last chunk for testing
    X_test = X[train_test_split:,:]
    y_test = y[train_test_split:]

            # NOTE: to use keras's RNN LSTM module our input must be reshaped to [samples, window size, stepsize] 
    X_train = np.asarray(np.reshape(X_train, (X_train.shape[0], window_size, 1)))
    X_test = np.asarray(np.reshape(X_test, (X_test.shape[0], window_size, 1)))

    lstm_model.fit(X_train, y_train, epochs=100, batch_size=20, verbose=0)

In [ ]:
#καλουμε την απο πανω συνάρτηση για train και test 
#μετά την καλουμε για τα testcases αφου πρώτα έχουμε καλεσει την window

In [18]:
test_case1 = [10 ,13 ,6 ,3 ]
test_case2 = [1 ,13, 6 ,9 ,13, 12 ,8 ,4 ,1]
test_case3 = [19, 19 ,39, 24 ,26, 21,16 ,13 ,14, 2,0, 11, 5, 7, 6, 3 ]
test_case4 = [10 ,59 ,84 ,126 ,163 ,174 ,195 ,210 ,196, 204 ,187 ,188 ,200, 173, 180, 157, 136 ,133 ]
test_case5 = [10 ,34 ,35 ,49, 73, 64, 68, 44, 34, 32 ,14 ]
test_case6 = [5 ,31 ,31 ,55 ,89 ,99 ,142 ,152 ,152]
test_case7 = [3,11,17,44,47,74,94,99,126,142,131]
test_case8 = [1,4,18,42,43,32,37,80,50,49,41,53,39,38,39,35,33,30,20]
test_case9 = [12,7,22,15,31,31,24,27,26,31,28,22]
test_case10 = [12,36,45,45,49,59,58,41]

In [42]:
test_cases = []
#test_cases.append(test_case1) only for window_size = 1
test_cases.append(test_case2)
test_cases.append(test_case3)
test_cases.append(test_case4)
test_cases.append(test_case5)
test_cases.append(test_case6)
test_cases.append(test_case7)
test_cases.append(test_case8)
test_cases.append(test_case9)
test_cases.append(test_case10)

In [102]:
#error of predict για y_train και y_test:
inversed_X = []
inversed_y = []
inversed_y = scaler.inverse_transform(y_train)

for i in X_train:
    inversed_X.append(scaler.inverse_transform(i))
inversed_X = np.asarray(inversed_X)
X = np.asarray(np.reshape(inversed_X,(inversed_X.shape[0], window_size, 1)))
train_prediction = prediction(X, inversed_y )

inversed_X = []
inversed_y = []
inversed_y = scaler.inverse_transform(y_test)

for i in X_test:
    inversed_X.append(scaler.inverse_transform(i))
inversed_X = np.asarray(inversed_X)
test_prediction = prediction(inversed_X, inversed_y )


In [101]:
#for the test cases:

predictionsOfTestCases = []

for i in range(1,9):
    X,y = window_transform_series(test_cases[i], window_size)
    X = np.asarray(np.reshape(X,(X.shape[0], window_size, 1)))
    temp = prediction(X,y)
    predictionsOfTestCases.append(temp)

